# Notebook 4ου εργαστηρίου
Πριν προχωρήσετε, εκτελέστε τα επόμενα δύο κελιά.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Ο αποκωδικοποιητής 2-σε-4
Στο επόμενο κελί εμφανίζεται μια πιθανή υλοποίηση του **αποκωδικοποιητή 2-σε-4** με πύλες (λογικές πράξεις).

In [ ]:
class Decoder2to4 extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(2.W))
    val y = Output(UInt(4.W))
  })
  
  val t0 = ~io.a(0) & ~io.a(1)
  val t1 = io.a(0) & ~io.a(1)
  val t2 = ~io.a(0) & io.a(1)
  val t3 = io.a(0) & io.a(1)
    
  io.y := Cat(t3,Cat(t2,Cat(t1,t0)))
}

Ακολουθεί ο έλεγχος ορθότητας για κάθε έναν από τους 4 συνδυασμούς εισόδου.

In [ ]:
test(new Decoder2to4()) { c =>
  c.io.a.poke("b00".U)
  c.io.y.expect("b0001".U)

  c.io.a.poke("b01".U)
  c.io.y.expect("b0010".U)

  c.io.a.poke("b10".U)
  c.io.y.expect("b0100".U)

  c.io.a.poke("b11".U)
  c.io.y.expect("b1000".U)
}
println("SUCCESS!!")

### Περιγραφή συμπεριφοράς με τη δομή when..elsewhen..otherwise
Ο ίδιος αποκωδικοποιητής 2-σε-4 με τη δομή `when`.

Παρατηρήστε ότι η ισότητα ελέγχεται με το `===` (τριπλό ίσον)!

In [ ]:
class Decoder2to4 extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(2.W))
    val y = Output(UInt(4.W))
  })
  
  when(io.a==="b00".U) {
    io.y := "b0001".U   
  }.elsewhen(io.a==="b01".U) {
    io.y := "b0010".U   
  }.elsewhen(io.a==="b10".U) {
    io.y := "b0100".U   
  }.otherwise {
    io.y := "b1000".U         
  }
}

### Περιγραφή συμπεριφοράς με τη δομή switch
Καλό είναι να μπαίνει πάντα μια αρχική τιμη (safeguard).

In [ ]:
class Decoder2to4 extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(2.W))
    val y = Output(UInt(4.W))
  })
  
  io.y := 0.U  // safeguard 
  switch(io.a) {
    is("b00".U) { io.y := "b0001".U }
    is("b01".U) { io.y := "b0010".U }
    is("b10".U) { io.y := "b0100".U }
    is("b11".U) { io.y := "b1000".U }
  }
}

## Ο πολυπλέκτης 4-σε-1
Στο επόμενο κελί εμφανίζεται μια πιθανή υλοποίηση του **πολυπλέκτη 4-σε-1** με πύλες (λογικές πράξεις).

In [ ]:
class Mux4to1 extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(1.W))
    val sel = Input(UInt(2.W))
  })
    
  val p0 = io.in(0) & ~io.sel(0) & ~io.sel(1)
  val p1 = io.in(1) & io.sel(0) & ~io.sel(1)
  val p2 = io.in(2) & ~io.sel(0) & io.sel(1)
  val p3 = io.in(3) & io.sel(0) & io.sel(1)
  io.out := p0 | p1 | p2 | p3
}

Ο έλεγχος ορθότητας εξετάζει για κάθε συνδυασμό του `io.sel` αν επιλέγεται η κατάλληλη είσοδος, για κάθε πιθανή τιμή εισόδου `io.in`.

In [ ]:
test(new Mux4to1()) { c =>
  for (s <- 0 to 3) {
    c.io.sel.poke(s.U)
    for (i <- 0 to 15) {
      c.io.in.poke(i.U)
      val mask = 1 << s
      c.io.out.expect(if ((i&mask)!=0) 1.U else 0.U)
    }
  }
}
println("SUCCESS!!")

### Ασκηση: περιγραφή συμπεριφοράς με το when
Περιγράψτε τον ίδιο **πολυπλέκτη 4-σε-1** με τη δομή `when`.

Ελέγξτε το αποτέλεσμα με το `test` που δόθηκε προηγουμένως.

In [ ]:
class Mux4to1 extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(1.W))
    val sel = Input(UInt(2.W))
  })
  
  // ο κ΄ώδικάς σας εδώ
  //when...
}

### Άσκηση: περιγραφή πολυπλέκτη με το switch
Περιγράψτε τον **πολυπλέκτη 4-σε-1** με τη δομή `switch`.

Ελέγξτε το αποτέλεσμα με το `test` που δόθηκε προηγουμένως.

In [ ]:
class Mux4to1 extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(1.W))
    val sel = Input(UInt(2.W))
  })
    
  // ο κώδικάς σας εδώ
  //switch...
}